### Controlar el acceso de red a S3 desde su VPC utilizando puntos finales de VPC

#### Problema
Los recursos dentro de su VPC deben ser capaces de acceder sólo a un cubo S3 específico. Además, este tráfico de S3 no debe atravesar Internet por razones de seguridad y para mantener bajos los costes de ancho de banda.

#### Solución
Creará un punto final de la VPC de gateway para S3, lo asociará con una tabla de rutas y personalizará su documento de políticas.

<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781492092599/files/assets/awsc_0213.png" width="500">

In [1]:
import boto3
import json
import time

region_aws = 'us-east-1'

ec2 = boto3.resource('ec2', region_name=region_aws)
ec2_client = boto3.client('ec2', region_name=region_aws)

In [2]:
# create VPC
vpc = ec2.create_vpc(CidrBlock='174.17.0.0/16')
vpc.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC"}])
vpc.wait_until_available()

In [9]:
# Crear routable
routetable_1 = vpc.create_route_table()
routetable_2 = vpc.create_route_table()
routetable_1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbookVPC-RT-1"}])
routetable_2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbookVPC-RT-2"}])

[ec2.Tag(resource_id='rtb-0f6dc00c3dac3aa18', key='Name', value='AWSCookbookVPC-RT-2')]

In [4]:
# Cree dos subredes:
subnet_1 = ec2.create_subnet(
    CidrBlock='174.17.0.0/24', 
    VpcId=vpc.id,
    AvailabilityZone='us-east-1a'
)

subnet_2 = ec2.create_subnet(
    CidrBlock='174.17.1.0/24', 
    VpcId=vpc.id,
    AvailabilityZone='us-east-1b'
)

subnet_1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-SN-Private-1"}])
subnet_2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-SN-Private-2"}])

[ec2.Tag(resource_id='subnet-02db86da144e2b3c9', key='Name', value='AWSCookbook-SN')]

In [11]:
# Asociar las tablas de rutas con las subred:
routetable_1.associate_with_subnet(SubnetId = subnet_1.id)
routetable_2.associate_with_subnet(SubnetId = subnet_2.id)

ec2.RouteTableAssociation(id='rtbassoc-0e9925ee59cec6c2f')

In [23]:
import random
# generate random number
random_number_1 = random.randint(100, 10000)
random_number_2 = random.randint(100, 1000)

In [25]:
# Crear un bucket de S3
s3 = boto3.resource('s3')
s3_name = 'awscookbookbucket-' + str(random_number_1) + '-' + str(random_number_2)

try:
    bucket = s3.create_bucket(
        Bucket=s3_name,
    )
except Exception as e:
    print(e)
else:
    print('Bucket created: {} '.format(bucket.name))

Bucket awscookbookbucket-2874-472 created


In [18]:
print("Crear una instancia EC2 con permisos SSM")
print("En la terminal ejecute:")
print("    python create_ec2_ssm.py --vpc {}".format(vpc.id))

Crear una instancia EC2 con permisos SSM
En la terminal ejecute:
    python create_ec2_ssm.py --vpc vpc-0c42dd2366c78bfdb


Cree un gateway-endpoint en su VPC y asocie el endpoint con las tablas de rutas aisladas:


In [20]:
gateway_endpoint = ec2_client.create_vpc_endpoint(
    VpcId=vpc.id,
    ServiceName='com.amazonaws.{}.s3'.format(region_aws),
    VpcEndpointType='Gateway',
    RouteTableIds=[
        routetable_1.id,
        routetable_2.id
    ]
)


Cree un archivo de política de punto final de plantilla llamado policy.json con el siguiente contenido. Esto se utiliza para limitar el acceso sólo al bucket de S3 que creó en los pasos de preparación:

In [30]:
policy_endpoint = {
  "Statement": [
    {
      "Sid": "RestrictToOneBucket",
      "Principal": "*",
      "Action": [
        "s3:GetObject",
        "s3:PutObject"
      ],
      "Effect": "Allow",
      "Resource": ["arn:aws:s3:::" + s3_name + "/",
                   "arn:aws:s3:::" + s3_name + "/*"]
    }
  ]
}

Modificar el documento de políticas del endpoint. Las políticas del endpoint limitan o restringen los recursos a los que se puede acceder a través del endpoint de la VPC:

In [35]:
ec2_client.modify_vpc_endpoint(
    VpcEndpointId=gateway_endpoint['VpcEndpoint']['VpcEndpointId'],
    PolicyDocument=json.dumps(policy_endpoint)
)

{'Return': True,
 'ResponseMetadata': {'RequestId': 'f5426a43-5fb9-4e2e-bd2e-6a53b618df36',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f5426a43-5fb9-4e2e-bd2e-6a53b618df36',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'accept-encoding',
   'content-type': 'text/xml;charset=UTF-8',
   'transfer-encoding': 'chunked',
   'date': 'Mon, 10 Oct 2022 16:29:38 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

#### Comprobaciones de validación

Imprime el nombre de tu S3 Bucket para que puedas referirte a él cuando estés conectado a tu Instancia EC2:

In [36]:
print(s3_name)

awscookbookbucket-2874-472


Conéctese a la instancia EC2 mediante el gestor de sesiones SSM 

In [37]:
print("En la terminal ejecute:")
print("    python create_ec2_ssm.py --last")

En la terminal ejecute:
    python create_ec2_ssm.py --last


En la instancia ejecute:

In [ ]:
export AWS_DEFAULT_REGION=$(curl \
--silent http://169.254.169.254/latest/dynamic/instance-identity/document \
| awk -F'"' ' /region/ {print $4}')

In [ ]:
BUCKET=$(aws ssm get-parameters \
    --names "awscookbookbucket" \
     --query "Parameters[*].Value" --output text)